# Tutorial 5: Adjoint Algorithmic Differentiation (AAD)

## How to use this tutorial

  * Select "run all cells" on this notebook from the Run menu in Jupyter notebook or Jupyter
    lab. This step will produce intermediate data output and charts.
  * Some cells print out a url, which you can click on and bring up an interactive web UI to
    visualize the graph data.
  * In the unlikely event that the notebook becomes irresponsive, you can try "Restart
    Kernel" from the Kernel menu, then run individual cells one by one using `Shift+Enter`.
  * Some tutorials use local clusters consisting of multiple processes to mimic the effects
    of graph distribution over a remote cluster. By default, these local clusters
    automatically stop after idling for 15 minutes to conserve CPU and memory resources. You
    will need to rerun the entire notebook if your local cluster stopped due to inactivity.
  * Additional resources (video demos & blogs) are available at http://juliustech.co.
  * To report any issues, get help or request features, please raise an issue at
    https://github.com/JuliusTechCo/JuliusGraph/issues.

## Introduction

The main topic of this tutorial is adjoint algorithmic differentiation (AAD), it is
a powerful technique to compute 1st order derivative quickly and exactly using chain rules.
AAD offers a significant performance gain, sometime thousands of times, comparing to the
traditional finite difference method (i.e. bump and recalculation) for computing first
order derivatives. AAD is particularly relevant for quantitative finance applications, as
the 1st order derivatives are widely used for hedging and risk management.

The traditional approach to implementing AAD requires a tape to record the forward
calculations, then the tape is run in reverse for the backward AD. This approach is memory
intensive and does not scale well to large applications or systems. A large system
consists of many software components running concurently on multiple computers, it is not
feasible to record a consistent tape across multiple computers.

Julius offers an easy and efficient way to implement AAD for large distributed systems.
Julius use the computational DAG to cache the value and keep track of the dependency of
the primal calculations, then the entire DAG is run in reverse for the AAD calculation.
Both primal and backward AD run can be fully distributed by Julius Graph Engine for high
performance and scalability, all of this is achieved without using the memory
intensive tape recordings.

AAD is a rich and complex topic, this tutorial is only a very high level introduction
to Julius AAD capabilities. Please reach out to us at (info@juliustech.co) for a more
in depth demo if you are interested in learning the full capabilities of Julius AAD.

Similar to the quickstart, we start with a simple Fibonacci sequence as the example.

## Quantom

We have introduced the type `Datom` in the quickstart tutorial. `Quantom` is another
important subtype of `Atom`, it stands for quantitative atom. ```Quantum``` is the main
work horse for numerical computations with AAD. Unlike Dataom that can take any data
types as inputs and output, the inputs and outputs of Quantoms must be vectors or
matrices of `Float64` (double precision floating point numbers). Non-numerical datatypes
would not make sense for AAD, which is the primary reason to use a `Quantom` type.

In this tutorial, we will not go into the details of the `Quantom` interface or
how to implement new `Quantom` types. Instead, we will use existing `Quantom` implementations
provied in Julius distribution to illustrate the AAD capabilities.

Julius provides a rich set of `Quantom`  libraries for the most common numerical algorithms,
including root searching, interpolation, and common stochastic processes etc. All the
Julius `Quantom` implementations come with full AAD support and have been extensively
tested.

In [ ]:
# disable the display of information logging
using Base.CoreLogging
disable_logging(CoreLogging.Info)

using GraphEngine: RuleDSL, GraphVM
using GraphEngine.RuleDSL
using GraphIO

# start data server for web UI
gss = Dict{String,RuleDSL.AbstractGraphState}()
port = GraphVM.drawdataport()
@async GraphVM.startresponder(gss, port);

The following few rules defines the Fibonacci sequence using the rule syntax. These rules
shows the type and value based polymorphism. Multiple rules of different types can be
defined for the same rule name, and at run time, the best matching rule is invoked.

These definitions are very similar to those we have seen in the quick start tutorials.
The only difference here is that the `RuleDSL.WegithedSum` quantum is used.

In [ ]:
@addrules series begin
    fib(x::Float64) = Alias(fib(Int(floor(x))))
    fib(n::Int) = RuleDSL.Alias(fib(n, Val(n <= 1)))
    fib(n::Int, isend::Val{false}) = begin
        RuleDSL.WeightedSum[[1.0; 1.0]](
            fib(n - 1, Val(n <= 2)), fib(n - 2, Val(n <= 3))
        )
    end
    fib(n::Int, isend::Val{true}) = RuleDSL.Constant[[fill(Float64(n), 1)]]()
end

config = RuleDSL.Config();

The following two functions override the nodelabel function for the customized display of
graph nodes. Users can freely override them to customize the text label display of the
computation graph.

In [ ]:
import GraphEngine.RuleDSL: nodelabel

# override displays
function nodelabel(::RuleDSL.AbstractGraphState, r::RuleDSL.RuleNode)
    hdr = "$(r.ns).$(r.op[1])"
    typestr(x) = x.args[2]
    typ = join(typestr.(r.op[2]), ", ")

    return hdr * "($typ)"
end

function nodelabel(::RuleDSL.AbstractGraphState, ref::RuleDSL.NodeRef)
    hdr = "$(ref.ns).$(ref.name)"
    ps = join(simplerepr.(ref.params), ", ")

    return "$hdr($ps)"
end

`RuleDSL.NumericalData` is a sub type of `GraphData` type that provides run time support to
the adjoint algorithm differentiation (AAD). The second parameter to `NumericalData` specifies
the default $\vec x$ in the AAD output of $\frac{\partial{\vec y}}{\partial {\vec x}}$.
Here $\vec y$ is is specified by the unique has id of `hash(fib5)`:

In [ ]:
fib5 = @ref series.fib(5.2)
fib0 = @ref series.fib(0, Val(true))
fib1 = @ref series.fib(1, Val(true))

cs = RuleDSL.NumericalData(config, Set([fib0, fib1]));
gs = RuleDSL.createlocalgraph(config, cs);

# primal calculation
RuleDSL.calcfwd!(gs, Set([fib5]));
# backward AD calculation
RuleDSL.calcback!(gs, Set([hash(fib5)]));

We can retrieve the results of $\vec y$ and the 1st order derivatives from AAD.

In [ ]:
y = RuleDSL.getys(gs, hash(fib5))
dydx0 = RuleDSL.getyds(gs, hash(fib0), hash(fib5))
dydx1 = RuleDSL.getyds(gs, hash(fib1), hash(fib5))

println("fib5 =", y[1])
println("dfib5_dfib0 =", dydx0[1])
println("dfib5_dfib1 =", dydx1[1])

The corresponding computation graph is shown below, and you can click the link below to
see it in an interactive web UI.

In [ ]:
svg = GraphIO.postlocalgraph(gss, deepcopy(gs), port, true; key="fib5");
display("image/svg+xml", svg)

## Caching

One of the main challenges in AAD implementation is its memory consumption. The reverse AD
calculation in AAD requires the primal calculation results to be cached. In the traditional
tape based approach, the memory required for caching the primal calculation results
can grow beyond the size of available physical memory, causing AAD execution to fail for
large problems. Advanced AAD techniques, such as checkpointing, are needed to limit the
memory consumption for large problems, but these advanced techniques requires additional
coding efforts thus further complicates the AAD implementation.

Julius Graph Engine automatically caches all the intermediate results in the computational
graph. For very large problems, Julius can automatically distribute the computation graph
across multiple computers, and leverage all of their physical memories. Therefore, Julius
can access practically an unlimited amount of memory when running in distributed mode,
therefore it is no longer subjected to the the physical memory limit of any single computer.

In addition, Julius only caches the primal results at the node level, as opposed to the
individual arithmetic operator level as in the traditional AAD taping implementation.
As a result, Julius caching is much more memory efficient for complex problems. Julius
effectively implemented automatic checkpointing for every individual `Quantum`.  An
`Quantom` object in each individual node can perform a heavy and complex calculation with
AAD, and the intermediate primal results within individual quantom do not need to be cached.

With Julius automatic caching of all intermediate computation results in the graph, any
incremental computation will automatically re-use existing values in the cache, instead of
being recomputing them from scatch. The following cells shows that when computing `fib(10)`,
all the nodes up to `fib(5)` from previous step is re-used.

In [ ]:
fib10 = @ref series.fib(10)
# primal calculation
GraphVM.calcfwd!(gs, Set([fib10]));
# backward AD calculation
RuleDSL.calcback!(gs, Set([hash(fib10)]));

y = RuleDSL.getys(gs, hash(fib10))
df10df0 = RuleDSL.getyds(gs, hash(fib0), hash(fib10))
df10df1 = RuleDSL.getyds(gs, hash(fib1), hash(fib10))

println("fib10 =", y[1])
println("dfib10_dfib0 =", df10df0[1])
println("dfib10_dfib1 =", df10df1[1])

The corresponding computation graph is shown below:

In [ ]:
svg = GraphIO.postlocalgraph(gss, deepcopy(gs), port, true; key="fib10");
display("image/svg+xml", svg)

## Namespace Clone and Override

Rules are organized within individual namespaces in Julius RuleDSL, for example the `series`
is a namespace in the rules declaration above.
One important feature of Julius is that the rules can be cloned to different namespace and
overridden. This facilitates comparisons between the old and new versions of rules, making
change management and impact analysis much easier.

For example, if we update the above Fibonacci sequence definition to take random vectors as
fib(0) and fib(1) instead of scalar 0 and 1, we can create a clone of the namespace "series"
 and override the corresponding rules.

In [ ]:
@clone series => vecs

@addrules vecs begin
    fib(n::Int, isend::Val{true}) = RuleDSL.Constant[[randn(10)]]()
end

In [ ]:
vfib10 = @ref vecs.fib(10)
# primal calculation
@time GraphVM.calcfwd!(gs, Set([vfib10]));

y = RuleDSL.getys(gs, hash(vfib10))
println("vfib10 =", y[1])

We can see an interactive side by side comparison of the two versions of the calculation,
before and after changing the initial values to random vectors, by clicking the URL below
to bring up Julius' interactive web UI.

In [ ]:
svg = GraphIO.postlocalgraph(gss, deepcopy(gs), port, true; key="vecs");
display("image/svg+xml", svg)

Julius AAD implementation supports multiple sensitivity views from the same primal
calculation. A sensitivity view is a Jacobian matrix
$\frac{\partial \vec y}{\partial \vec x}$ where $\vec y$ and $\vec x$ can be any connected
node in the graph.

The following few cells shows the computation of different sensitivity views witout
repeating the primal calculation. Please note
that the AAD output of the vectorized version of Fibonacci sequence is a full Jacobian
matrix.

In [ ]:
v10 = @ref vecs.fib(10)
v0 = @ref vecs.fib(0, Val(true))
v1 = @ref vecs.fib(1, Val(true))

# backward AD
RuleDSL.calcback!(gs, Set(hash.([v10])), Set([v0; v1]));

dv10_dv0 = RuleDSL.getyds(gs, hash(v0), hash(v10))
dv10_dv1 = RuleDSL.getyds(gs, hash(v1), hash(v10))

println("fib_v10 = ", RuleDSL.getys(gs, hash(v10))[1])
println("dv10_dv0 = ", dv10_dv0[1])
println("dv10_dv1 = ", dv10_dv1[1])

In [ ]:
v8 = @ref vecs.fib(8, Val(false))
v2 = @ref vecs.fib(2, Val(false))
v3 = @ref vecs.fib(3, Val(false))

# backward AD with a different sensitivity view
RuleDSL.calcback!(gs, Set(hash.([v8])), Set([v2; v3]));

dv8_dv2 = RuleDSL.getyds(gs, hash(v2), hash(v8))
dv8_dv3 = RuleDSL.getyds(gs, hash(v3), hash(v8))

println("fib_v8 =", RuleDSL.getys(gs, hash(v8))[1])
println("dv8_dv0 =", dv8_dv2[1])
println("dv8_dv1 =", dv8_dv3[1])

## Distributed AAD

With `Quantum` and `NumericalData`, Julius allows AAD to be easily implemented
for large and complex applications. In this section, we show a real world AAD use case
of pricing a portfolio of derivatives with distributed graph.

We first start a local cluster consisting of 3 worker processes to mimic a physical
cluster.

In [ ]:
using JuliusApps: QFin

# date grid for pricing the portfolio
dgrid = QFin.dategrid(80; nmonth=3)
config = RuleDSL.newconfig(QFin.QFinConfig(dgrid), :project=>"PV/AAD");

# the balancer defines different strategies to distribute the graph computation
# CopyRVs is the most efficient strategy for AAD
balancer = GraphVM.CopyRVs();
my_domain = GraphVM.mydomain()

# draw a port number to start the local cluster esrvice
clusterport = GraphVM.drawdataport();

In [ ]:
# start a local master service at the given port

GraphVM.startlocalmasterservice(clusterport, 3);

We then create a portfolio of 1000 trades, with 10 desks, each consisting of 10 trading books
with 10 trades of Swap, Swaptions, CDS or cross currency swaps etc.

In [ ]:
ndesk, nbook, ntrade = 10, 10, 10
nd = RuleDSL.getconfig(config, :nd)
asofs = collect(0:4:nd-1)
nodes, apv, aads = QFin.createbook(config, asofs, ndesk, nbook, ntrade);
book = apv.params[1];

# risk view, defining the sensitivities to compute, ie, the x in dy/dx
rvs = Set([RuleDSL.@ref fac.riskview(book)]);

In [ ]:
cs = RuleDSL.NumericalData(config, rvs)
gs = GraphVM.RemoteGraphProxy(config, my_domain => clusterport, balancer, cs, true)
GraphVM.wait4clusterinit(gs)
GraphVM.rpccall(gs.rpc, :workerstatus)

In [ ]:
GraphVM.@remote_eval gs begin
    using JuliusApps, GraphIO
    using GraphEngine: RuleDSL, GraphVM
end

GraphVM.waitcheckstatus(gs, RuleDSL.getconfig(config, :project));

In [ ]:
jobs = Set{Vector{RuleDSL.NodeRef}}([k.first] for k in nodes);
GraphVM.createremotegraph(gs, jobs, Set{Symbol}());

In [ ]:
cid = UInt(0)
eid = hash(apv)
GraphVM.calcfwd!(gs, cid, Set(eid))
GraphVM.calcback!(gs, eid, asofs)

GraphVM.waitcheckstatus(gs, RuleDSL.getconfig(config, :project));

# show the stats of resulting graph
stats = GraphVM.rpccall(gs, GraphIO.graphstats, UInt(1), UInt(1))
stats[:cnt]

The resulting graph has about 30,000 nodes and cannot be displayed on a single chart, we only
show the top portion instead. The full AAD results can be visualized by clicking the URL
below to enter the interactive web UI, then select the AAD link on the right panel.

In [ ]:
svg = GraphIO.postremotegraph(gs, port, false; maxnode=UInt(120));
display("image/svg+xml", svg)

The following call shows the computation time for each stage of pricing the 1,000 derivative
portfolio. The computational time is reported in seconds for graph creation, primal
calculation, AAD calculation and final clean up.

In this pricing example, there are about 800 different risk factors, and the $\vec y$ is an
pricing vector for 20 annual time steps under forcasted market evolution for the next 20
years. Each of the risk factor has 80 quarterly sample interval, therefore the total number
of individual sensitivities are roughly 800 x 20 x 80 = 1.28MM, all of which are computed
via AAD in about 15 seconds (depending on the hardware), which is extremely fast.

In [ ]:
GraphVM.rpccall(gs, :displaychecks)

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*